<a href="https://colab.research.google.com/github/Michel-p16/PDS-Project/blob/capstone_korbi/DTS_final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Distilbert zur Evaluierung


In [1]:
!pip install transformers datasets
#1. Daten laden + filtern (SINGLE SELECT here)
import json
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

from sklearn.metrics import classification_report

import os

import numpy as np
from sklearn.utils.class_weight import compute_class_weight
import torch
import torch.nn as nn

with open("generated_responses_few_errors.json", "r") as file:
    dataset = json.load(file)

# Filtere das Dataset nach Fragen mit dem Typ "SINGLE_SELECT"
filtered_dataset = [example for example in dataset if example["type"] == "SINGLE_SELECT"]



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
#2. Daten formatieren

def convert_to_distilbert_format(data):
    formatted_data = []
    all_labels = []  # Labels für Single-Select

    for example in data:
        question = example["question"]
        answers = example["answers"]

        for answer in answers:
            text = answer["text"]
            label = answer.get("label")

            # Kontext aus der Antwort extrahieren (da er nicht separat vorhanden ist)
            context = text

            if label is not None:
                all_labels.append(label)
            formatted_data.append({
                "question": question,
                "context": context,  # Kontext ist hier die Antwort selbst
                "answers": {"text": [text], "answer_start": [0]}, # answer_start ist 0, da Kontext = Antwort
                "label": label
            })

    # Label-Encodierung für Single-Select
    label_encoder = LabelEncoder()
    label_encoder.fit(list(set(all_labels)))
    for example in formatted_data:
        if example["label"] is not None:
            example["label"] = label_encoder.transform([example["label"]])[0]

    return formatted_data, label_encoder  # Gib formatted_data und label_encoder zurück


# Hier den Funktionsaufruf außerhalb der Funktion platzieren:
formatted_dataset, label_encoder = convert_to_distilbert_format(filtered_dataset)

In [3]:
#3. Daten splitten 80% Training 20% Evaluation

train_data_formatted, eval_data_formatted = train_test_split(formatted_dataset, test_size=0.2, random_state=42)

print(f"Trainingsdaten: {len(train_data_formatted)}")
print(f"Evaluationsdaten: {len(eval_data_formatted)}")

Trainingsdaten: 560
Evaluationsdaten: 140


In [4]:
import random

# Anzahl der Beispiele, die ausgegeben werden sollen
num_samples = 10

# Zufällige Beispiele aus den Trainingsdaten auswählen
random_indices_train = random.sample(range(len(train_data_formatted)), num_samples)

print("Formatierte Trainingsdaten:")
for index in random_indices_train:
    example = train_data_formatted[index]
    print(f"Beispiel {index + 1}:")
    print(f"  Frage: {example['question']}")
    print(f"  Kontext: {example['context']}")
    print(f"  Antworten: {example['answers']}")
    print(f"  Label: {example['label']}")  # Überprüfen Sie hier den Label-Typ
    print("-" * 20)

# Zufällige Beispiele aus den Evaluationsdaten auswählen
random_indices_eval = random.sample(range(len(eval_data_formatted)), num_samples)

print("\nFormatierte Evaluationsdaten:")
for index in random_indices_eval:
    example = eval_data_formatted[index]
    print(f"Beispiel {index + 1}:")
    print(f"  Frage: {example['question']}")
    print(f"  Kontext: {example['context']}")
    print(f"  Antworten: {example['answers']}")
    print(f"  Label: {example['label']}")  # Überprüfen Sie hier den Label-Typ
    print("-" * 20)

Formatierte Trainingsdaten:
Beispiel 14:
  Frage: May we process your data?
  Kontext: 
  Antworten: {'text': [''], 'answer_start': [0]}
  Label: 1
--------------------
Beispiel 430:
  Frage: May we process your data?
  Kontext: No, I do not consent to my data being processed.
  Antworten: {'text': ['No, I do not consent to my data being processed.'], 'answer_start': [0]}
  Label: 0
--------------------
Beispiel 63:
  Frage: May we process your data?
  Kontext: Yes, you may process my data.
  Antworten: {'text': ['Yes, you may process my data.'], 'answer_start': [0]}
  Label: 2
--------------------
Beispiel 432:
  Frage: May we process your data?
  Kontext: Yes, you may process my data.
  Antworten: {'text': ['Yes, you may process my data.'], 'answer_start': [0]}
  Label: 2
--------------------
Beispiel 66:
  Frage: May we process your data?
  Kontext: Yes, No, or Unsure.
  Antworten: {'text': ['Yes, No, or Unsure.'], 'answer_start': [0]}
  Label: 0
--------------------
Beispiel 90:
  

In [5]:
# 4. Labels vorbereiten
# Extrahiere die Labels für den LabelEncoder
train_labels = [example["label"] for example in train_data_formatted if example["label"] is not None]

# Erstellen und Anpassen des LabelEncoders
label_encoder = LabelEncoder()
label_encoder.fit(train_labels)

# Funktion zum Transformieren der Labels
def transform_label(example):
    if example["label"] is not None:
        example["label"] = label_encoder.transform([example["label"]])[0]
    return example

# Transformiere die Labels in den formatierten Daten
train_data_formatted = [transform_label(example) for example in train_data_formatted]

In [6]:
# 5. Datasets erstellen
train_dataset = Dataset.from_pandas(pd.DataFrame(train_data_formatted))
eval_dataset = Dataset.from_pandas(pd.DataFrame(eval_data_formatted))


# Überprüfen Datenmenge
print(f"Anzahl der Trainingsdaten: {len(train_dataset)}")
print(f"Anzahl der Evaluationsdaten: {len(eval_dataset)}")


Anzahl der Trainingsdaten: 560
Anzahl der Evaluationsdaten: 140


In [7]:
# 6. Vorverarbeitung

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_function(examples):
    return tokenizer(
        examples["question"], examples["context"],
        padding=True, truncation=True, max_length=128
    )


train_dataset = train_dataset.map(preprocess_function, batched=True)
eval_dataset = eval_dataset.map(preprocess_function, batched=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/560 [00:00<?, ? examples/s]

Map:   0%|          | 0/140 [00:00<?, ? examples/s]

In [8]:
#test ob inbalanced answers/labels
from collections import Counter

train_labels = [sample['label'] for sample in train_data_formatted]
eval_labels = [sample['label'] for sample in eval_data_formatted]

print("Trainingsdaten Klassenverteilung:", Counter(train_labels))
print("Evaluationsdaten Klassenverteilung:", Counter(eval_labels))


Trainingsdaten Klassenverteilung: Counter({2: 228, 0: 226, 1: 106})
Evaluationsdaten Klassenverteilung: Counter({2: 54, 0: 53, 1: 33})


In [9]:
# 7.Training start
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=len(label_encoder.classes_))  # num_labels anpassen

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    learning_rate=2e-5,
    weight_decay=0.01,
    evaluation_strategy="epoch",  # Evaluation nach jeder Epoche
    save_strategy="epoch",  # Speichern nach jeder Epoche
    logging_dir="./logs",
    load_best_model_at_end=True, # bestes Modell am Ende laden,
    metric_for_best_model="accuracy",  # Wähle die Metrik, die das beste Modell definiert
    greater_is_better=True,  # Für Metriken wie Accuracy, Precision, etc
    report_to="none",  # Deaktiviert WandB
    logging_steps=10,  # Logge Fortschritte alle 10 Schritte
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,  # Evaluationsdatensatz hinzufügen
    compute_metrics=compute_metrics,  # Metrikenfunktion hinzufügen

)

trainer.train()

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.046900,0.047200,1.000000,1.000000,1.000000,1.000000
2,0.010200,0.007809,1.000000,1.000000,1.000000,1.000000
3,0.006500,0.004973,1.000000,1.000000,1.000000,1.000000
4,0.005600,0.004389,1.000000,1.000000,1.000000,1.000000


TrainOutput(global_step=280, training_loss=0.11761502756604127, metrics={'train_runtime': 716.347, 'train_samples_per_second': 3.127, 'train_steps_per_second': 0.391, 'total_flos': 15068435316480.0, 'train_loss': 0.11761502756604127, 'epoch': 4.0})

In [10]:
#8. Evaluation nach Training

# 1. Laden des besten Modells
checkpoints = [ckpt for ckpt in os.listdir("./results") if ckpt.startswith("checkpoint")]
latest_checkpoint = max(checkpoints, key=lambda x: int(x.split('-')[1]))
model_path = f"./results/{latest_checkpoint}"
print(f"Using model from: {model_path}")

# 2. Evaluation Dataset vorbereiten
eval_df = pd.DataFrame(eval_data_formatted)  # Verwende deine formatierten Evaluationsdaten
eval_texts = eval_df["question"].tolist()
eval_labels = eval_df["label"].tolist()  # Extrahiere die tatsächlichen Labels

# 3. Tokenizer laden
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Hugging Face Dataset erstellen
eval_dataset = Dataset.from_dict({"text": eval_texts, "label": eval_labels})
eval_dataset = eval_dataset.map(preprocess_function, batched=True)

# 4. Modellvorhersagen
predictions = trainer.predict(eval_dataset)
predicted_labels = np.argmax(predictions.predictions, axis=1)

# 5. Vorhersagen in DataFrame einfügen
eval_df["predicted_label"] = predicted_labels

# 6. Labels zurückmappen (falls codiert)
eval_df["true_label"] = eval_df["label"]
eval_df["predicted_label_decoded"] = label_encoder.inverse_transform(eval_df["predicted_label"])
eval_df["true_label_decoded"] = label_encoder.inverse_transform(eval_df["true_label"])

# 7. Berechnung der Metriken
# Konvertiere die Klassenlabels in Strings
target_names = [str(label) for label in label_encoder.classes_]
# Klassifikationsbericht erstellen und ausgeben
print("Klassifikationsbericht:")
print(classification_report(eval_df["true_label"], eval_df["predicted_label"], target_names=target_names))





Using model from: ./results/checkpoint-280


Map:   0%|          | 0/140 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Klassifikationsbericht:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        53
           1       0.24      1.00      0.38        33
           2       0.00      0.00      0.00        54

    accuracy                           0.24       140
   macro avg       0.08      0.33      0.13       140
weighted avg       0.06      0.24      0.09       140



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [11]:
# 9. Manuelle Eingabe Test

# 1. Laden des Modells und Tokenizers
model_path = f"./results/{latest_checkpoint}"  # Stelle sicher, dass das Modell korrekt geladen wird
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Mapping der Labels
label_mapping = {
    0: "No",
    1: "No Preference",
    2: "Yes"
}

# 2. Manuelle Eingabe
text = input("Gib einen Text ein, den das Modell klassifizieren soll: ")

# 3. Vorverarbeitung der Eingabe
inputs = tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=512)

# 4. Modellvorhersage
model.eval()  # Setzt das Modell in den Evaluationsmodus
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()

# 5. Decodierung der Vorhersage
predicted_label = label_mapping[predicted_class]  # Nutze das Mapping

# 6. Ausgabe
print(f"Der eingegebene Text wurde als '{predicted_label}' klassifiziert.")


Gib einen Text ein, den das Modell klassifizieren soll: yes please
Der eingegebene Text wurde als 'Yes' klassifiziert.


In [ ]:
#evaluationsergebnisse
print(eval_df[["question", "predicted_label"]])

In [21]:
#10. Daten laden + filtern (MULTI SELECT here)

from sklearn.preprocessing import MultiLabelBinarizer

with open("generated_responses_few_errors.json", "r") as file:
    dataset = json.load(file)

# Filtere das Dataset nach Fragen mit dem Typ "MULTI_SELECT"
filtered_dataset_multi = [example for example in dataset if example["type"] == "MULTI_SELECT"]

In [37]:
# 11. Daten formatieren für MULTI_SELECT + Split + vorbereiten

def convert_to_multi_select_format(data, mlb):  # Übergebe den MultiLabelBinarizer als Argument
    formatted_data = []

    for example in data:
        question = example["question"]
        answers = example["answers"]

        for answer in answers:
            text = answer["text"]
            labels = answer.get("label")

            if isinstance(labels, list):
                formatted_data.append({
                    "text": text,
                    "labels": mlb.transform([labels])[0]  # Verwende den übergebenen MultiLabelBinarizer
                })

    return formatted_data  # Gib nur formatted_data zurück, da der MultiLabelBinarizer bereits angepasst ist


# Alle eindeutigen Labels aus beiden Datensätzen sammeln
all_labels = []
for dataset in [filtered_dataset, filtered_dataset_multi]:
    for example in dataset:
        for answer in example["answers"]:
            labels = answer.get("label")
            if isinstance(labels, list):
                all_labels.extend(labels)

# MultiLabelBinarizer mit allen eindeutigen Labels erstellen und anpassen
multi_label_binarizer = MultiLabelBinarizer(classes=list(set(all_labels)))
multi_label_binarizer.fit([[label] for label in all_labels])

# Funktionsaufrufe mit dem angepassten MultiLabelBinarizer
formatted_dataset = convert_to_multi_select_format(filtered_dataset, multi_label_binarizer)
formatted_multi_dataset = convert_to_multi_select_format(filtered_dataset_multi, multi_label_binarizer)

# Daten splitten 80% Training 20% Evaluation

train_multi_data_formatted, eval_multi_data_formatted = train_test_split(formatted_multi_dataset, test_size=0.2, random_state=42)

print(f"Trainingsdaten: {len(train_multi_data_formatted)}")
print(f"Evaluationsdaten: {len(eval_multi_data_formatted)}")



Trainingsdaten: 318
Evaluationsdaten: 80


In [35]:
for example in filtered_dataset_multi[:5]:
    print(f"Question: {example['question']}")
    print(f"Answers: {example['answers']}")


Question: Which products are you interested in?
Answers: [{'text': "I don't care", 'label': 'No Preference', 'timestamp': '2025-01-21T11:29:56.288797'}, {'text': "I'm interested in Notion, JTS, AX100, AKW100, and MY-SYSTEM.", 'label': ['Notion', 'JTS', 'AX100', 'AKW100', 'MY-SYSTEM'], 'timestamp': '2025-01-21T11:30:02.206290'}, {'text': "I'm interested in Notion, JTS, and the AX100.", 'label': ['Notion', 'JTS', 'AX100'], 'timestamp': '2025-01-21T11:30:07.899016'}, {'text': "Hmm, I'm really looking at MY-SYSTEM and the AX100 right now.  JS EcoLine is also on my radar, but Notion's a bit further down the list for me.  So those four, really.", 'label': ['MY-SYSTEM', 'AX100', 'JS EcoLine', 'Notion'], 'timestamp': '2025-01-21T11:30:13.917440'}, {'text': "Oh, I'm looking at the AX100 and the AKW100.  Trying to decide between the two, actually.  They both seem pretty good, but I'm not sure which one's best for my needs.", 'label': ['AX100', 'AKW100'], 'timestamp': '2025-01-21T11:30:19.937298'

In [34]:
import random

num_samples = 3  # Anzahl der Beispiele, die ausgegeben werden sollen

# Zufällige Beispiele aus den Trainingsdaten auswählen
random_indices_train = random.sample(range(len(train_multi_data_formatted)), num_samples)

print("Zufällige Beispiele aus den Trainingsdaten:")
for index in random_indices_train:
    example = train_multi_data_formatted[index]
    print(f"Beispiel {index + 1}:")
    print(f"  Text: {example['text']}")  # Zugriff auf den Text der Antwort
    # Konvertiere example['labels'] in ein NumPy-Array
    labels_array = np.array([example['labels']])
    decoded_labels = multi_label_binarizer.inverse_transform(labels_array)[0]
    print(f"  Labels (dekodiert): {decoded_labels}")
    print("-" * 20)

# Zufällige Beispiele aus den Evaluationsdaten auswählen
random_indices_eval = random.sample(range(len(eval_multi_data_formatted)), num_samples)

print("\nZufällige Beispiele aus den Evaluationsdaten:")
for index in random_indices_eval:
    example = eval_multi_data_formatted[index]
    print(f"Beispiel {index + 1}:")
    print(f"  Text: {example['text']}")  # Zugriff auf den Text der Antwort
    # Konvertiere example['labels'] in ein NumPy-Array
    labels_array = np.array([example['labels']])
    decoded_labels = multi_label_binarizer.inverse_transform(labels_array)[0]
    print(f"  Labels (dekodiert): {decoded_labels}")
    print("-" * 20)

Zufällige Beispiele aus den Trainingsdaten:
Beispiel 36:
  Text: Definitely MY-SYSTEM, and I'm also curious about the JS EcoLine – how does its eco-friendliness compare to the others?
  Labels (dekodiert): ('JTS', 'MY-SYSTEM', 'JS EcoLine')
--------------------
Beispiel 24:
  Text: Primarily interested in AKW100, MY-SYSTEM, and Notion.  Possibly AX100.
  Labels (dekodiert): ('JTS', 'Notion', 'AKW100', 'MY-SYSTEM', 'JS EcoLine', 'AX100')
--------------------
Beispiel 117:
  Text: I'm interested in the AX100 and JTS products.
  Labels (dekodiert): ('JTS', 'AX100')
--------------------

Zufällige Beispiele aus den Evaluationsdaten:
Beispiel 38:
  Text: Notion, primarily for project management and note-taking.
  Labels (dekodiert): ('Notion',)
--------------------
Beispiel 11:
  Text: Definitely MY-SYSTEM and Notion,  I'm also curious about the JTS and AKW100 specs.
  Labels (dekodiert): ('JTS', 'Notion', 'AKW100', 'MY-SYSTEM', 'JS EcoLine', 'AX100')
--------------------
Beispiel 30:
  Tex

In [38]:
#12. Datasets erstellen (Multiselect)

train_multi_dataset = Dataset.from_pandas(pd.DataFrame(train_multi_data_formatted))
eval_multi_dataset = Dataset.from_pandas(pd.DataFrame(eval_multi_data_formatted))

# Überprüfen Datenmenge (Multi-Select)
print(f"Anzahl der Trainingsdaten (Multi-Select): {len(train_multi_dataset)}")
print(f"Anzahl der Evaluationsdaten (Multi-Select): {len(eval_multi_dataset)}")

Anzahl der Trainingsdaten (Multi-Select): 318
Anzahl der Evaluationsdaten (Multi-Select): 80


In [39]:
# 13. Vorverarbeitung (Multi-Select)

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_function_multi(examples):
    return tokenizer(
        examples["text"], # Verwenden Sie "text" als Eingabe
        padding=True,
        truncation=True,
        max_length=128
    )

train_multi_dataset = train_multi_dataset.map(preprocess_function_multi, batched=True)
eval_multi_dataset = eval_multi_dataset.map(preprocess_function_multi, batched=True)

Map:   0%|          | 0/318 [00:00<?, ? examples/s]

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

In [40]:
#14. label verteilung test

from collections import Counter

# Extrahiere die Labels aus den Trainingsdaten
train_multi_labels = [example["labels"] for example in train_multi_data_formatted]

# Zähle die Vorkommen der einzelnen Labels
label_counts = Counter()
for labels in train_multi_labels:
    for i, label in enumerate(labels):
        if label == 1:  # Wenn das Label vorhanden ist
            label_counts[i] += 1  # Erhöhe den Zähler für dieses Label

# Gib die Label-Verteilung aus
print("Trainingsdaten (Multi-Select) Klassenverteilung:")
for label_index, count in label_counts.items():
    print(f"Label {label_index}: {count} mal")

# Dasselbe für die Evaluationsdaten
eval_multi_labels = [example["labels"] for example in eval_multi_data_formatted]
eval_label_counts = Counter()
for labels in eval_multi_labels:
    for i, label in enumerate(labels):
        if label == 1:
            eval_label_counts[i] += 1

print("\nEvaluationsdaten (Multi-Select) Klassenverteilung:")
for label_index, count in eval_label_counts.items():
    print(f"Label {label_index}: {count} mal")

Trainingsdaten (Multi-Select) Klassenverteilung:
Label 2: 188 mal
Label 5: 199 mal
Label 0: 186 mal
Label 1: 201 mal
Label 3: 176 mal
Label 4: 196 mal

Evaluationsdaten (Multi-Select) Klassenverteilung:
Label 3: 49 mal
Label 0: 52 mal
Label 1: 44 mal
Label 2: 43 mal
Label 4: 47 mal
Label 5: 52 mal


In [49]:
import torch
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# Definiere die benutzerdefinierte Loss-Funktion
def multi_label_loss_fn(logits, labels):
    # Reshape labels to match logits shape
    labels = labels.view(logits.shape)
    loss = F.binary_cross_entropy_with_logits(logits, labels.float())
    return loss

# 15. Training start (Multi-Select)

# num_labels anpassen an die Anzahl der eindeutigen Labels
model_multi = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=multi_label_binarizer.classes_.shape[0]
)

class MultiLabelTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):  # Add num_items_in_batch
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss = multi_label_loss_fn(logits, labels)
        return (loss, outputs) if return_outputs else loss

# Metriken für Multi-Label-Klassifikation
def compute_metrics_multi(pred):
    labels = pred.label_ids
    preds = torch.sigmoid(torch.tensor(pred.predictions))  # Sigmoid-Aktivierung für Multi-Label
    preds = (preds > 0.5).int()  # Schwellenwert für die Klassifizierung

    # Berechne die Metriken
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average="micro"
    )
    acc = accuracy_score(labels, preds)

    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}


training_args_multi = TrainingArguments(
    output_dir="./results_multi",  # Anderer Output-Ordner
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=4,
    learning_rate=2e-5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs_multi",  # Anderer Logging-Ordner
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    report_to="none",
    logging_steps=10,
)

trainer_multi = MultiLabelTrainer(  # Verwende die benutzerdefinierte Trainer-Klasse
    model=model_multi,
    args=training_args_multi,
    train_dataset=train_multi_dataset,
    eval_dataset=eval_multi_dataset,
    compute_metrics=compute_metrics_multi,
)

trainer_multi.train()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ValueError: Expected input batch_size (8) to match target batch_size (48).